In [392]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [393]:
data = pd.read_csv("/kaggle/input/titanic/train.csv")

In [394]:
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
X = data[features]
y = data.Survived #series

In [395]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [396]:
#deal with nan
X_train["Age"] = X_train["Age"].fillna(X_train["Age"].mean())
X_test["Age"] = X_test["Age"].fillna(X_test["Age"].mean())
X_train["Embarked"] = X_train["Embarked"].fillna(str(X_train["Embarked"].mode()[0]))
X_test["Embarked"] = X_test["Embarked"].fillna(str(X_test["Embarked"].mode()[0]))

In [397]:
#deal with str features
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(X_train["Sex"])
X_train["Sex"] = le.transform(X_train["Sex"])
X_test["Sex"] = le.transform(X_test["Sex"])
le.fit(X_train["Embarked"])
X_train["Embarked"] = le.transform(X_train["Embarked"])
X_test["Embarked"] = le.transform(X_test["Embarked"])

In [398]:
from sklearn.ensemble import RandomForestRegressor
my_rtr = RandomForestRegressor(random_state=1) #max_depth = 100
my_rtr.fit(X_train, y_train)
test_preds = my_rtr.predict(X_test)

In [399]:
submission_lst = []
for i in range(len(test_preds)):
    #print(y_test.iloc[i] == (1 if test_preds[i] > 0.5 else 0))
    submission_lst.append(1 if test_preds[i] > 0.5 else 0)

In [400]:
count = 0
for i in range(len(submission_lst)):
    if(submission_lst[i] == y_test.iloc[i]):
        count+= 1
print(count / len(test_preds))

0.8044692737430168


In [401]:
test_csv_data = pd.read_csv('/kaggle/input/titanic/test.csv')
# SHOULD USE COPY bc usually you have train_test_split to make X_train
# an obvious copy (doesn't deal with pandas weirdness). The warning is
# fine to get bc you don't care if test_csv_data gets modified, but to
# keep the program from worrying just do a copy.
X_real = test_csv_data[features].copy()
# This is the line that normally makes it an explicit copy:
#X_real, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [402]:
X_real.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S


In [403]:
missing_val_count_by_column = (X_real.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Age     86
Fare     1
dtype: int64


In [404]:
print(X_train.head())
print(X_real.head(10))

     Pclass  Sex        Age  SibSp  Parch     Fare  Embarked
140       3    0  29.745184      0      2  15.2458         0
439       2    1  31.000000      0      0  10.5000         2
817       2    1  31.000000      1      1  37.0042         0
378       3    1  20.000000      0      0   4.0125         0
491       3    1  21.000000      0      0   7.2500         2
   Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0       3    male  34.5      0      0   7.8292        Q
1       3  female  47.0      1      0   7.0000        S
2       2    male  62.0      0      0   9.6875        Q
3       3    male  27.0      0      0   8.6625        S
4       3  female  22.0      1      1  12.2875        S
5       3    male  14.0      0      0   9.2250        S
6       3  female  30.0      0      0   7.6292        Q
7       2    male  26.0      1      1  29.0000        S
8       3  female  18.0      0      0   7.2292        C
9       3    male  21.0      2      0  24.1500        S


In [405]:
#str(X_real["Embarked"].mode()[0])
#print(X_real["Age"].fillna(X_real["Age"].mean()))
X_real["Fare"].fillna(X_real["Fare"].mean(), inplace = True) 

In [406]:
X_real["Age"].fillna(X_real["Age"].mean(), inplace = True) 

In [407]:
#CONFUSING
# print ([X_real[col].dtype for col in X_real.columns])
# for col in X_real.columns:
#     #if X_real[col].dtype == 0:
#     print(X_real[col].dtype)

In [408]:
#deal with 
#print(X_real[age])

#print(X_real["Sex"].shape, len(le.transform(X_real["Sex"])))
for col in X_real.columns:
    #print(X_real[col].dtype)
    if X_real[col].dtype == object:
        print(col)
        X_real[col] = le.fit_transform(X_real[col])

#X_real = le.fit_transform(X_real)
#le.fit(X_real["Embarked"])
#X_real["Embarked"] = le.transform(X_real["Embarked"])

Sex
Embarked


In [409]:
submission_preds = my_rtr.predict(X_real)
submission_lst_real = []
for i in range(len(submission_preds)):
    submission_lst_real.append(1 if submission_preds[i] > 0.5 else 0)

In [410]:
sub_sr = pd.DataFrame({"PassengerId": test_csv_data.PassengerId,
                  "Survived": submission_lst_real})

In [411]:
missing_val_count_by_column = (X_real.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Series([], dtype: int64)


In [412]:
sub_sr.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,0


In [413]:
sub_sr.to_csv("submission.csv", index = False)